In [9]:
# !pip install azure-cognitiveservices-speech
# !pip install azure-search-documents
# !pip install openai
# !pip install fastapi uvicorn
# !pip install ipywidgets



In [8]:
import azure.cognitiveservices.speech as speechsdk
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
import ipywidgets as widgets
from IPython.display import display, Audio

In [16]:
# Azure Configuration
AZURE_SPEECH_KEY = "13Dmw7ZFBYRuwq7xyrFA8pzx3J2ZMDaO3vRWV883FtFm6Lgq5FtaJQQJ99BBACYeBjFXJ3w3AAAYACOGJDGr"
AZURE_SPEECH_REGION = "eastus"
AZURE_SEARCH_SERVICE = "testaisearchmctltfs030225"
AZURE_SEARCH_INDEX = "loan-documents-index"
AZURE_SEARCH_KEY = "X5ayCxEXO3KNEKDgUp6hn466Gd7iDsukNQujhRfXV2AzSeDNKv8a"
AZURE_OPENAI_KEY = "55FnF6s4J4OywbWkhBg75P6iFekRErJK3IYW40zlwD5EfUnFoH5MJQQJ99BBACHYHv6XJ3w3AAABACOGEtm9"
AZURE_OPENAI_ENDPOINT = "https://demoopenairesource.openai.azure.com/"
AZURE_DEPLOYMENT_NAME = "gpt-4o-realtime-preview"
AZURE_OPENAI_API_VERSION="2024-10-01-preview"


In [14]:
# Initialize Azure AI Clients
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version = AZURE_OPENAI_API_VERSION
    # deployment=AZURE_DEPLOYMENT_NAME
)

search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=AZURE_SEARCH_KEY
)

# Conversation Memory
conversation_history = []

In [17]:
def speech_to_text():
    speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SPEECH_REGION)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)
    
    print("Listening for input...")
    result = speech_recognizer.recognize_once()
    
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    elif result.reason == speechsdk.ResultReason.NoMatch:
        return "I didn't understand. Please try again."
    else:
        return "Speech recognition failed."


In [18]:
def search_knowledge_base(query):
    results = search_client.search(query, query_type=QueryType.SEMANTIC)
    docs = [doc['content'] for doc in results]
    return " ".join(docs[:3]) if docs else "No relevant information found."


In [19]:
def generate_response(user_input):
    conversation_history.append(f"User: {user_input}")
    
    context = search_knowledge_base(user_input)
    prompt = f"Context: {context}\nUser: {user_input}\nAI:"
    
    response = openai_client.completions.create(
        model=AZURE_DEPLOYMENT_NAME,
        prompt=prompt,
        max_tokens=200
    )
    
    ai_response = response.choices[0].text.strip()
    conversation_history.append(f"AI: {ai_response}")
    return ai_response


In [20]:
def text_to_speech(response_text):
    
    speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SPEECH_REGION)
    audio_config = speechsdk.audio.AudioOutputConfig(filename="response.wav")
    
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    synthesizer.speak_text_async(response_text).get()

    # Play audio in Jupyter Notebook
    return Audio("response.wav", autoplay=True)


In [23]:
def chat_loop():
    while True:
        user_input = speech_to_text()
        if "exit" in user_input.lower():
            print("Ending conversation.")
            break

        response = generate_response(user_input)
        print(f"Bot: {response}")
        text_to_speech(response)


In [24]:
@app.post("/chat")
async def chat():
    user_input = speech_to_text()
    response = generate_response(user_input)
    text_to_speech(response)
    return {"response": response}


NameError: name 'app' is not defined

In [22]:
# # Widgets for Jupyter Interaction
# start_button = widgets.Button(description="Start Voice Chat")
# output_area = widgets.Output()

# def on_button_click(b):
#     with output_area:
#         print("🎤 Listening...")
#         user_input = speech_to_text()
        
#         print(f"🗣️ User: {user_input}")
#         response = generate_response(user_input)
        
#         print(f"🤖 AI: {response}")
#         display(text_to_speech(response))

# start_button.on_click(on_button_click)

# display(start_button, output_area)


Button(description='Start Voice Chat', style=ButtonStyle())

Output()